In [2]:
pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [langchain]/7 [langgraph]
Note: you may need to restart the kernel to use updated packages.


In [171]:
import importlib
importlib.reload(config)

<module 'config' from '/Users/kumar/Desktop/Projects/fake_review_detection/config.py'>

In [172]:
import numpy as np
import pandas as pd
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from config import Config
from evaluate import evaluate_model

In [ ]:
# Load your CSV
df = pd.read_csv(Config.file_path) [350:450]
reviews = df["review"].tolist()

In [173]:
prompt = PromptTemplate(
    input_variables=["text"], # the placeholder to be replaced based on prompt template
    template=Config.zero_shot_prompt_template
)

# Load small model
llm = OllamaLLM(model=Config.model)

# Create chain using pipe operator (modern LangChain syntax)
chain = prompt | llm

In [139]:
# Classify each review
results = []
for r in reviews:
    result = chain.invoke({"text": r})
    # Clean the output - extract only "truthful" or "deceptive"
    label = result.strip().lower()
    # Extract first word if model adds extra text
    if " " in label:
        label = label.split()[0]
    # Remove any punctuation
    label = label.strip('.,!?;:')
    print(f"Raw output: {result} -> Cleaned: {label}")
    results.append(label)

Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Truthful -> Cleaned: truthful
Raw output: Truthful -> Clea

In [ ]:
accuracy, f1, conf_matrix = evaluate_model(df, results)

print("=" * 50)
print("ZERO-SHOT PROMPTING RESULTS")
print("=" * 50)
print("Accuracy:", accuracy)
print("\nF1 Score:", f1)
print("\nConfusion Matrix:")
print(conf_matrix)
print("=" * 50)

Accuracy: 0.46

F1 Score: 0.3894165535956581

Confusion Matrix:
[[40 10]
 [44  6]]


## One-Shot Prompting
Using one example to guide the model's classification

In [ ]:
one_shot_prompt = PromptTemplate(
    input_variables=["text"],
    template = Config.one_shot_prompt_template
)
# Create one-shot chain
one_shot_chain = one_shot_prompt | llm

In [132]:
# Classify each review
one_shot_results = []
for r in reviews:
    result = one_shot_chain.invoke({"text": r})
    # Clean the output - extract only "truthful" or "deceptive"
    label = result.strip().lower()
    # Extract first word if model adds extra text
    if " " in label:
        label = label.split()[0]
    # Remove any punctuation
    label = label.strip('.,!?;:')
    print(f"Raw output: {result} -> Cleaned: {label}")
    one_shot_results.append(label)

Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Truthful -> Cleaned: truthful
Raw output: Truthful -> Cleaned: truthful
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned

In [ ]:
accuracy_one_shot, f1_one_shot, conf_matrix_one_shot = evaluate_model(df, one_shot_results)

print("=" * 50)
print("ONE-SHOT PROMPTING RESULTS")
print("=" * 50)
print("Accuracy:", accuracy_one_shot)
print("\nF1 Score:", f1_one_shot)
print("\nConfusion Matrix:")
print(conf_matrix_one_shot)
print("=" * 50)

Accuracy: 0.5

F1 Score: 0.3658041603247082

Confusion Matrix:
[[48  2]
 [48  2]]


## Few-Shot Prompting
Using multiple examples to guide the model's classification

In [175]:
few_shot_prompt = PromptTemplate(
    input_variables=["text"],
    template=Config.few_shot_prompt_template
)

# Create few-shot chain
few_shot_chain = few_shot_prompt | llm

In [121]:
# Classify reviews using few-shot prompting
few_shot_results = []
for r in reviews:
    result = few_shot_chain.invoke({"text": r})
    # Clean the output
    label = result.strip().lower()
    if " " in label:
        label = label.split()[0]
    label = label.strip('.,!?;:')
    print(f"Raw output: {result} -> Cleaned: {label}")
    few_shot_results.append(label)

Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Truthful -> Cleaned: truthful
Raw output: Truthful -> Cleaned: truthful
Raw output: Truthful. -> Cleaned: truthful
Raw output: Truthful. -> Cleaned: truthful
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Truthful. -> Cleaned: truthful
Raw output: Truthful. -> Clea

In [ ]:
accuracy_few_shot, f1_few_shot, conf_matrix_few_shot = evaluate_model(df, few_shot_results)

print("=" * 50)
print("FEW-SHOT PROMPTING RESULTS")
print("=" * 50)
print("Accuracy:", accuracy_few_shot)
print("\nF1 Score:", f1_few_shot)
print("\nConfusion Matrix:")
print(conf_matrix_few_shot)
print("=" * 50)

Accuracy: 0.5

F1 Score: 0.47916666666666674

Confusion Matrix:
[[35 15]
 [35 15]]


## Compare All Approaches
Compare zero-shot, one-shot, and few-shot results

In [141]:
# Compare all approaches
comparison_df = pd.DataFrame({
    'Approach': ['Zero-Shot', 'One-Shot', 'Few-Shot'],
    'Accuracy': [accuracy, accuracy_one_shot, accuracy_few_shot],
    'F1 Score': [f1, f1_one_shot, f1_few_shot]
})

print("=" * 60)
print("COMPARISON OF ALL PROMPTING APPROACHES")
print("=" * 60)
print(comparison_df.to_string(index=False))
print("=" * 60)

# Find best approach
best_idx = comparison_df['Accuracy'].idxmax()
best_approach = comparison_df.loc[best_idx, 'Approach']
best_accuracy = comparison_df.loc[best_idx, 'Accuracy']
print(f"\nBest Approach: {best_approach} (Accuracy: {best_accuracy:.4f})")

COMPARISON OF ALL PROMPTING APPROACHES
 Approach  Accuracy  F1 Score
Zero-Shot      0.46  0.389417
 One-Shot      0.50  0.365804
 Few-Shot      0.50  0.479167

Best Approach: One-Shot (Accuracy: 0.5000)
